In [ ]:
HTML(read(open("style.html"), String))

## Basic Game

Das Modul `Chess.jl` stellt eine Implementation des Schachspiels für die Sprache Julia bereit. Dieses kann gegebenenfalls über den Packagemanager `Pkg` installiert werden.

In [ ]:
using Pkg
Pkg.add("Chess")

In [ ]:
using Chess

Ein Schachspiel besteht aus einem abwechselnden Durchführen jeweils genau eines Zuges. Besondere Konventionen wie zum Beispiel Rochaden zählen hierbei als ein Zug.

Hierbei folgt das Spielen eines Schachspiels stets der folgenden Logik:

ai_turn = false

while !isterminal(game)

    move = next_move(game, ai_turn ? ai : player)
    domove!(game, move)
    ai_turn = !ai_turn
    
end     

Die Funktion `next_move` : (`board`, `entity`) -> `move` bildet auf den nächsten durchzuführenden Zug ab. Hierbei wird die Implementation der Funktion über Multiple-Dispatch mit dem `entity` Attribut bestimmt. Dieses welchselt mit der Partei am Zug.

### Entity Player

Die Spieler-Implementation (Methode) der `next_move` Funktion setzt einer Benutzer-Eingabe voraus, welche in Julia mithilfe der `readline`(`stream`) Funktion erreicht werden kann. 

In [ ]:
input = readline(stdin)
print(input)

Wie bereits zu erkennen ist, kann der Benutzer jeden beliebigen Text als Eingabe vormulieren, auch solche, die nicht einer validen Zugeingabe entsprechen. Um einer solchen fehlerhaften Eingabe entgegen zu wirken, definieren wir die folgende Funktion:

In [ ]:
function validate_userinput(input::String)
    if match(r"^[a-hA-H][1-8][a-hA-H][1-8]$", input) === nothing
        return "Invalid input '$(input)'"
    end
    return nothing
end

Der reguläre Ausdruck `^[a-h][1-8][a-h][1-8]$` akzeptiert hierbei nur Eingabetexte, welche aus zwei Feldern bestehen: Start & Ende. Da es sich bei der X- und Y-Indizierung auf einem Schachbrett um Koordinaten im Bereich [a-h] und [a-8] handelt, werden diese zusätzlich auf die entsprechenden Bereiche begrenzt. Je nach Status des Spiels ist eine Angabe des Startfelden theoretisch redundant, jedoch entschieden wir uns, die verbose Form als Eingabenorm beizubehalten.

Falls die Eingabe der `validate_userinput` Funktion einen syntaktisch validen Zug aufweist, so gibt diese `nothing` zurück. Der Rückgabewert ist als eine Art Felher zu interpretieren. Ist die Eingabe invalid, so wird ein String mit der Fehlernachricht zurückgegeben.

Des weiteren ist zu prüfen, ob es sich bei dem syntaktisch validen Zug um einen tatsächlich legalen und durchführbaren Zug handelt. Beispielsweise könnte der Benutzer in der Startposition den Zug `"e2f2"` eingeben, welcher dem Bauer instruktieren würde, um ein Feld nach rechts zu gehen. Dieser Zug ist laut Spielregeln nicht möglich und muss daher unterbunden werden. Hierzu definieren wir die folgende Funktion:

In [ ]:
function validate_legal_move(board::Board, move::Move)
    all_legal_moves = moves(board)
    if move ∉ all_legal_moves
        return("Illegal move '$(move)'")
    end
    return nothing
end

Mithilfe der `moves(board)` Funktion können wir alle legalen Züge auflisten. Anschließend ist zu prüfen, ob sich der Zugkandidat in der entsprechenden Liste befindet. In Julia kann dies durch die respektiven Operatoren `∈` und `∉` durchgeführt werden. Die `validate_legal_move` Funktion gibt bei einem Fehler ebenfalls einen String zurück, andersweitig `nothing`.

Mit den beiden Prüffunktionen können wir anschließend die `next_move` Funktion für einen Player definieren:

In [ ]:
struct Player end

function next_move(board, _::Player)
    move = nothing
    while true
        userinput = readline(stdin)
        if userinput == "exit" || userinput == "reset"
            return userinput
        end
        error = validate_userinput(userinput)
        if error != nothing
            println(error)
            continue
        end
        move = movefromstring(userinput)
        error = validate_legal_move(board, move)
        if error != nothing
            println(error)
            continue
        end
        break
    end
    return move
end

Als zusätzlich valide Eingaben definieren wir die Worte `exit` und `reset`, welche gegebenenfalls das Spiel vorzeitig beenden oder zurücksetzten können.

## Game Implementation

Mit allen bereits implementieren Komponenten lässt sich eine erste, lauffähige Durchführung eines Schachspiels implementieren:

In [ ]:
function play_game(ai, player = Player(), game = SimpleGame(), ai_turn = false)
    print_game(game)
    
    while !isterminal(game)
        move = next_move(board(game), ai_turn ? ai : player)
        if move == "exit"
            IJulia.clear_output()
            return board(game)
        end
        if move == "reset"
            return play_game(ai)
        end
        domove!(game, move)
        print_game(game)
        ai_turn = !ai_turn
    end
    
    return isdraw(game) ? "Draw!" : ai_turn ? "Checkmate! You win!" : "Checkmate! You lost!"
end

Nachdem ein Spiel beendet wurde, gibt die Funktion das Ergebnis es Spiels zurück.

Um die Ausgabe während des Spiels zu vereinfachen, definieren wir die folgende Funktion, welche die aktuelle Ausgabezelle löscht und das Spielbrett erneut ausgibt. Konvertiert man ein `Chess.jl`- Schachbrett zu HTML, so wird dieses in einer anschaulichen Form ausgerendert.

In [ ]:
function print_game(game)
    IJulia.clear_output()
    html = HTML(IJulia.html(board(game)))
    IJulia.display(html)
end

Als letzte Komponente für ein vollständiges Schachspiel fehlt ein KI-Gegner. Hierzu importieren wir die `RandomAI`.

In [ ]:
# using Pkg
# Pkg.add("NBInclude")

In [ ]:
using NBInclude
@nbinclude("RandomAI.ipynb")

In [ ]:
play_game(RandomAI())

***

by Florian Stach and Luc Kaiser

Last updated: 30/11/2022

***